In [1]:
import pandas as pd
import re

# Funzione per estrarre i temi da V2Themes
def extract_themes(v2themes):
    if pd.isna(v2themes):  # Gestisce valori mancanti
        return []
    themes = v2themes.split(';')  # Divide i temi separati da ';'
    themes = [re.sub(r',.*', '', theme) for theme in themes]  # Rimuove il peso (tutto dopo la virgola)
    return themes

df_themes = pd.read_csv('/Volumes/CHERRY_SSD/gkg_myanmar_filtered.csv')

df_themes['theme'] = df_themes['V2Themes'].apply(extract_themes)
df_themes = df_themes['theme'].explode('theme')  # Crea una riga per ogni tema
# Conta i temi unici
theme_counts = df_themes.value_counts().reset_index()
theme_counts.columns = ['theme', 'count']

theme_counts.to_csv('../Data/Themes.csv', index=False)

In [7]:
import pandas as pd

# Carica i file come dataframe
df_themes = pd.read_csv('../Data/Themes.csv')
df_v2themes_count = pd.read_csv('../Data/v2themes_count.csv')

# Esegui il join utilizzando il metodo merge
df_merged = pd.merge(df_themes, df_v2themes_count, on='theme', how='left')
df_merged = df_merged[["theme","count","indicator"]]
df_merged['count'] = df_merged['count'].astype(int)
# Visualizza il dataframe risultante
df_merged.to_csv("../Data/merged_themes.csv", index = False)


In [8]:
# Esegui un left join e filtra le righe senza corrispondenza
df_only_in_themes = df_themes[~df_themes['theme'].isin(df_v2themes_count['theme'])]

# Salva il risultato in un file CSV
df_only_in_themes.to_csv('../Data/themes_not_in_v2themes.csv', index=False)

In [16]:
df_2 = pd.read_csv('../Data/themes_not_in_v2themes.csv')
df_themes = pd.read_csv('../Data/Themes.csv')
df_merged = pd.merge(df_themes, df_2, on='theme', how='left')
df_merged.to_csv("../Data/merged2_themes.csv", index = False)

In [18]:
import pandas as pd

# Carica i dataframe
df_2 = pd.read_csv('../Data/themes_not_in_v2themes.csv')
df_themes = pd.read_csv('../Data/Themes.csv')

# Esegui il merge con un left join
df_merged = pd.merge(df_themes, df_2, on='theme', how='left', suffixes=('_themes', '_df2'))

# Seleziona l'attributo indicator corretto
df_merged['indicator'] = df_merged['indicator_df2'].combine_first(df_merged['indicator_themes'])

# Rimuovi le colonne temporanee (indicator_themes e indicator_df2)
df_merged.drop(columns=['indicator_themes', 'indicator_df2','count_df2'], inplace=True)

# Salva il risultato in un file CSV
df_merged.to_csv("../Data/merged2_themes.csv", index=False)

In [20]:
df_merged = pd.read_csv("../Data/merged2_themes.csv")

temi = df_merged[df_merged['indicator'].notnull()]



In [22]:
temi.to_csv("../Data/temi.csv", index=False)

In [26]:
temi.to_excel("../Data/themes.xlsx")